In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import cdsapi 

# viz
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from cartopy.io import shapereader
import plotly.graph_objects as go

import glob
import sys
import os

# creds
sys.path.append('/Users/max/Deep_Sky')
from creds import CDS_UID, CDS_API_key

In [ ]:
# point to local UNSEEN open dir to import retrieve and preprocess
sys.path.append('/Users/max/Deep_Sky/GitHub/UNSEEN-open-Deep-Sky')

import src.cdsretrieve as retrieve
import src.preprocess as preprocess

c = cdsapi.Client()

In [ ]:
SEAS5 = preprocess.merge_SEAS5(folder ='../../data/UNSEEN/heatwave/SEAS5/', target_months = [5,6,7])
SEAS5

In [20]:
ERA5 = xr.open_mfdataset('../../data/UNSEEN/heatwave/ERA5/ERA5_????.nc', combine='by_coords')
ERA5

<xarray.Dataset>
Dimensions:    (latitude: 12, longitude: 21, time: 135)
Coordinates:
  * longitude  (longitude) float32 -111.1 -110.1 -109.1 ... -93.07 -92.07 -91.07
  * latitude   (latitude) float32 44.1 43.1 42.1 41.1 ... 36.1 35.1 34.1 33.1
  * time       (time) datetime64[ns] 1979-05-01 1979-06-01 ... 2023-07-01
Data variables:
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(3, 12, 21), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(3, 12, 21), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-03-28 23:08:27 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [ ]:
# Define a function to convert Kelvin to Celsius
def kelvin_to_celsius(temp_k):
    return temp_k - 273.15

In [ ]:
def season_mean(ds, years, calendar='standard'):
    # Make a DataArray with the number of days in each month, size = len(time)
    month_length = ds.time.dt.days_in_month

    # Calculate the weights by grouping by 'time.season'
    weights = month_length.groupby('time.year') / month_length.groupby('time.year').sum()

    # Test that the sum of the weights for each season is 1.0
    np.testing.assert_allclose(weights.groupby('time.year').sum().values, np.ones(years))

    # Calculate the weighted average
    return (ds * weights).groupby('time.year').sum(dim='time', min_count = 3)

In [ ]:

ERA5_yearly = season_mean(ERA5, len(pd.to_datetime(ERA5.time.values).year.unique()))
ERA5_yearly

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Convert the 't2m' variable to a pandas DataFrame
df = ERA5_yearly['t2m'].to_dataframe().reset_index()

df_mean = df.groupby('year')['t2m'].mean().reset_index()

df_mean['t2m'] = df_mean['t2m'].apply(kelvin_to_celsius)

# Create a line plot of 't2m' over time
fig = go.Figure(data=go.Scatter(x=df_mean['year'], y=df_mean['t2m'], mode='lines'))

# Set plot title and labels
fig.update_layout(title='Temperature over time', xaxis_title='Year', yaxis_title='Temperature')

# Show the plot
fig.show()

In [17]:
drought_area = [44.898, -111.066, 33.095, -90.631]
nebraska = [42.980712, -104.023496, 40.053028, -95.498106]
kansas = [40.002552, -102.045957, 36.993966, -94.641172]
oklahoma = [36.993966, -100.024473, 33.623964, -94.465391]
colorado = [41.004953, -109.011288, 37.011514, -102.023984]
wyoming = [45.011585, -111.076718, 41.004953, -104.056455]
iowa = [43.453090, -96.552792, 40.580763, -91.015684]
south_dakota = [45.920751, -103.979550, 42.940512, -96.552792]
missouri = [40.539030, -94.696103, 36.500995, -89.510557]

In [19]:
ERA5

NameError: name 'ERA5' is not defined

In [20]:
def filter_coordinates(xarray, latlon):
    mask = ((xarray.y >= latlon[2]) &
            (xarray.y <= latlon[0]) &
            (xarray.x >= latlon[1]) &
            (xarray.x <= latlon[3]))
    filtered_ds = xarray.where(mask, drop=True)
    return filtered_ds

In [ ]:
ne_ERA5 = filter_coordinates(ERA5, nebraska)
ks_ERA5 = filter_coordinates(ERA5, kansas)
ok_ERA5 = filter_coordinates(ERA5, oklahoma)
co_ERA5 = filter_coordinates(ERA5, colorado)
wy_ERA5 = filter_coordinates(ERA5, wyoming)
ia_ERA5 = filter_coordinates(ERA5, iowa)
sd_ERA5 = filter_coordinates(ERA5, south_dakota)
mo_ERA5 = filter_coordinates(ERA5, missouri)

In [ ]:
ne_ERA5_yearly = season_mean(ne_ERA5, len(pd.to_datetime(ne_ERA5.time.values).year.unique()))
ks_ERA5_yearly = season_mean(ks_ERA5, len(pd.to_datetime(ks_ERA5.time.values).year.unique()))
ok_ERA5_yearly = season_mean(ok_ERA5, len(pd.to_datetime(ok_ERA5.time.values).year.unique()))
co_ERA5_yearly = season_mean(co_ERA5, len(pd.to_datetime(co_ERA5.time.values).year.unique()))
wy_ERA5_yearly = season_mean(wy_ERA5, len(pd.to_datetime(wy_ERA5.time.values).year.unique()))
ia_ERA5_yearly = season_mean(ia_ERA5, len(pd.to_datetime(ia_ERA5.time.values).year.unique()))
sd_ERA5_yearly = season_mean(sd_ERA5, len(pd.to_datetime(sd_ERA5.time.values).year.unique()))
mo_ERA5_yearly = season_mean(mo_ERA5, len(pd.to_datetime(mo_ERA5.time.values).year.unique()))

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Define a function to convert Kelvin to Celsius
def kelvin_to_celsius(temp_k):
    return temp_k - 273.15

# Create an empty figure
fig = go.Figure()

# List of states and corresponding datasets
states = ['Nebraska', 'Kansas', 'Oklahoma', 'Colorado', 'Wyoming', 'Iowa', 'South Dakota', 'Missouri']
datasets = [ne_ERA5_yearly, ks_ERA5_yearly, ok_ERA5_yearly, co_ERA5_yearly, wy_ERA5_yearly, ia_ERA5_yearly, sd_ERA5_yearly, mo_ERA5_yearly]  

for state, ds in zip(states, datasets):

    df = ds['t2m'].to_dataframe().reset_index()

    df_mean = df.groupby('year')['t2m'].mean().reset_index()

    df_mean['t2m'] = df_mean['t2m'].apply(kelvin_to_celsius)

    # Add a line to the plot for this state
    fig.add_trace(go.Scatter(x=df_mean['year'], y=df_mean['t2m'], mode='lines', name=state))

# Set plot title and labels
fig.update_layout(title='Temperature over time', xaxis_title='Year', yaxis_title='Temperature')

# Show the plot
fig.show()

In [ ]:
# Create an empty figure
fig = go.Figure()

# List to hold DataFrames for each state
dfs = []

for state, ds in zip(states, datasets):
    # Convert the 't2m' variable to a pandas DataFrame
    df = ds['t2m'].to_dataframe().reset_index()

    df_mean = df.groupby('year')['t2m'].mean().reset_index()

    df_mean['t2m'] = df_mean['t2m'].apply(kelvin_to_celsius)

    # Calculate 5-year rolling average
    df_mean['t2m_rolling'] = df_mean['t2m'].rolling(window=5).mean()

    # Calculate percentage change to previous 5 years
    df_mean['t2m_pct_change'] = df_mean['t2m_rolling'].pct_change(periods=5)

    # Add state name column
    df_mean['state'] = state

    # Append DataFrame to list
    dfs.append(df_mean)

# Concatenate all DataFrames
df_all = pd.concat(dfs)

# Print the DataFrame
df_all.to_csv('temperature_change_by_state.csv')

In [31]:
tmax = xr.open_mfdataset('../../data/UNSEEN/heatwave/NASA/*.nc', combine='by_coords')
tmax

<xarray.Dataset>
Dimensions:                  (nv: 2, time: 6935, x: 284, y: 584)
Coordinates:
  * x                        (x) float32 -5.802e+06 -5.801e+06 ... -5.519e+06
  * y                        (y) float32 -3.9e+04 -4e+04 ... -6.21e+05 -6.22e+05
    lat                      (time, y, x) float32 dask.array<chunksize=(343, 343, 284), meta=np.ndarray>
    lon                      (time, y, x) float32 dask.array<chunksize=(343, 343, 284), meta=np.ndarray>
  * time                     (time) datetime64[ns] 2005-01-01T12:00:00 ... 20...
Dimensions without coordinates: nv
Data variables:
    yearday                  (time) int16 dask.array<chunksize=(365,), meta=np.ndarray>
    time_bnds                (time, nv) datetime64[ns] dask.array<chunksize=(365, 2), meta=np.ndarray>
    lambert_conformal_conic  (time) int16 -32767 -32767 -32767 ... -32767 -32767
    tmax                     (time, y, x) float32 dask.array<chunksize=(365, 584, 284), meta=np.ndarray>
Attributes:
    start_year:        2005
    source:            Daymet Software Version 4.0
    Version_software:  Daymet Software Version 4.0
    Version_data:      Daymet Data Version 4.0
    Conventions:       CF-1.6
    citation:          Please see http://daymet.ornl.gov/ for current Daymet ...
    references:        Please see http://daymet.ornl.gov/ for current informa...

In [13]:
# Create new variables for stress days and breakdown days
tmax['stress_days'] = tmax['tmax'] > 27.8
tmax['breakdown_days'] = tmax['tmax'] > 32.8

# Convert these to int for summing
tmax['stress_days'] = tmax['stress_days'].astype(int)
tmax['breakdown_days'] = tmax['breakdown_days'].astype(int)

# Group by year
tmax_yearly = tmax.resample(time='1Y').sum()

tmax_yearly

/Users/max/.pyenv/versions/3.12.1/lib/python3.12/site-packages/xarray/core/groupby.py:349: FutureWarning:

is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.



<xarray.Dataset>
Dimensions:                  (time: 19, x: 284, y: 584)
Coordinates:
  * time                     (time) datetime64[ns] 2005-12-31 ... 2023-12-31
  * x                        (x) float32 -5.802e+06 -5.801e+06 ... -5.519e+06
  * y                        (y) float32 -3.9e+04 -4e+04 ... -6.21e+05 -6.22e+05
Data variables:
    yearday                  (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    lambert_conformal_conic  (time) int64 -11959955 -11959955 ... -11959955
    tmax                     (time, y, x) float32 dask.array<chunksize=(1, 584, 284), meta=np.ndarray>
    stress_days              (time, y, x) int64 dask.array<chunksize=(1, 584, 284), meta=np.ndarray>
    breakdown_days           (time, y, x) int64 dask.array<chunksize=(1, 584, 284), meta=np.ndarray>

In [5]:
startyr = 1996
endyr = 2023

region = "na"
var = "tmax"
north=44.898
west=-111.066
east = -90.631
south = 33.095

In [47]:
for i in range(startyr, endyr + 1):
    print(startyr)
    print(i)
    if i % 4 == 0:
        !wget -O ../../data/UNSEEN/heatwave/NASA/{var}_{i}subset.nc "https://thredds.daac.ornl.gov/thredds/ncss/grid/ornldaac/2129/daymet_v4_daily_{region}_{var}_{i}.nc?var=lat&var=lon&var={var}&north={north}&west={west}&east={east}&south={south}&horizStride=50&time_start={i}-01-01T12:00:00Z&time_end={i}-12-30T12:00:00Z&timeStride=1&accept=netcdf"
    else:
        !wget -O ../../data/UNSEEN/heatwave/NASA/{var}_{i}subset.nc "https://thredds.daac.ornl.gov/thredds/ncss/grid/ornldaac/2129/daymet_v4_daily_{region}_{var}_{i}.nc?var=lat&var=lon&var={var}&north={north}&west={west}&east={east}&south={south}&horizStride=50&time_start={i}-01-01T12:00:00Z&time_end={i}-12-31T12:00:00Z&timeStride=1&accept=netcdf"

1996
1996
--2024-05-09 20:39:55--  https://thredds.daac.ornl.gov/thredds/ncss/grid/ornldaac/2129/daymet_v4_daily_na_tmax_1996.nc?var=lat&var=lon&var=tmax&north=44.898&west=-111.066&east=-90.631&south=33.095&horizStride=50&time_start=1996-01-01T12:00:00Z&time_end=1996-12-30T12:00:00Z&timeStride=1&accept=netcdf
Resolving thredds.daac.ornl.gov (thredds.daac.ornl.gov)... 160.91.19.90
Connecting to thredds.daac.ornl.gov (thredds.daac.ornl.gov)|160.91.19.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1416884 (1.4M) [application/x-netcdf]
Saving to: ‘../../data/UNSEEN/heatwave/NASA/tmax_1996subset.nc’

../../data/UNSEEN/h 100%[===================>]   1.35M   684KB/s    in 2.0s    

2024-05-09 20:45:59 (684 KB/s) - ‘../../data/UNSEEN/heatwave/NASA/tmax_1996subset.nc’ saved [1416884/1416884]

1996
1997
--2024-05-09 20:45:59--  https://thredds.daac.ornl.gov/thredds/ncss/grid/ornldaac/2129/daymet_v4_daily_na_tmax_1997.nc?var=lat&var=lon&var=tmax&north=44.898&west=-11

In [48]:
tmax = xr.open_mfdataset('../../data/UNSEEN/heatwave/NASA/tmax*.nc', combine='by_coords')
tmax

<xarray.Dataset>
Dimensions:                  (time: 12410, x: 2, y: 2)
Coordinates:
  * y                        (y) float32 309.0 -691.0
  * x                        (x) float32 -997.2 2.75
  * time                     (time) datetime64[ns] 1990-01-01T12:00:00 ... 20...
Data variables:
    lat                      (time, y, x) float32 dask.array<chunksize=(365, 2, 2), meta=np.ndarray>
    lambert_conformal_conic  (time) int16 -32767 -32767 -32767 ... -32767 -32767
    lon                      (time, y, x) float32 dask.array<chunksize=(365, 2, 2), meta=np.ndarray>
    tmax                     (time, y, x) float32 dask.array<chunksize=(365, 2, 2), meta=np.ndarray>
Attributes:
    start_year:          1990
    source:              Daymet Software Version 4.0
    Version_software:    Daymet Software Version 4.0
    Version_data:        Daymet Data Version 4.0
    Conventions:         CF-1.6
    citation:            Please see http://daymet.ornl.gov/ for current Dayme...
    references:          Please see http://daymet.ornl.gov/ for current infor...
    History:             Translated to CF-1.0 Conventions by Netcdf-Java CDM ...
    geospatial_lat_min:  35.345082756088495
    geospatial_lat_max:  45.421021821095096
    geospatial_lon_min:  -113.24934780298739
    geospatial_lon_max:  -99.95647587117966

In [49]:
tmax['stress_days'] = tmax['tmax'] > 30

tmax['stress_days'] = tmax['stress_days'].astype(int)

tmax_yearly = tmax.resample(time='1Y').sum()

tmax_yearly

/Users/max/.pyenv/versions/3.12.1/lib/python3.12/site-packages/xarray/core/groupby.py:349: FutureWarning:

is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.



<xarray.Dataset>
Dimensions:                  (time: 34, x: 2, y: 2)
Coordinates:
  * time                     (time) datetime64[ns] 1990-12-31 ... 2023-12-31
  * y                        (y) float32 309.0 -691.0
  * x                        (x) float32 -997.2 2.75
Data variables:
    lat                      (time, y, x) float32 dask.array<chunksize=(1, 2, 2), meta=np.ndarray>
    lambert_conformal_conic  (time) int64 -11959955 -11959955 ... -11959955
    lon                      (time, y, x) float32 dask.array<chunksize=(1, 2, 2), meta=np.ndarray>
    tmax                     (time, y, x) float32 dask.array<chunksize=(1, 2, 2), meta=np.ndarray>
    stress_days              (time, y, x) int64 dask.array<chunksize=(1, 2, 2), meta=np.ndarray>

In [56]:
stress_days_mean = tmax_yearly['stress_days'].mean(dim=['x', 'y'])

# Convert 'time' values to the number of years since the first year
time_values = (stress_days_mean['time'].values - stress_days_mean['time'].values[0]).astype('timedelta64[Y]').astype(int)

# Calculate the coefficients of the best fit line
coefficients = np.polyfit(time_values, stress_days_mean.values, 1)

# Generate y-values based on the best fit line
best_fit_line = coefficients[0] * time_values + coefficients[1]

# Create a line plot of 'stress_days' over time
fig = go.Figure()

# Add the original data
fig.add_trace(go.Scatter(x=stress_days_mean['time'].values, y=stress_days_mean.values, mode='lines', name='Number of Stress Days'))

# Add the best fit line
fig.add_trace(go.Scatter(x=stress_days_mean['time'].values, y=best_fit_line, mode='lines', name='Expected Stress Days'))

# Set plot title and labels
fig.update_layout(title='Stress Days (>30°C) over Time', xaxis_title='Year', yaxis_title='Stress Days')

# Show the plot
fig.show()

In [57]:
import plotly.io as pio

pio.write_image(fig, '../../figures/UNSEEN_heatwave/corn_stress_days.png')

# Save the figure as an HTML file
pio.write_html(fig, '../../figures/UNSEEN_heatwave/corn_stress_days.html')

In [51]:
tmax_yearly['stress_days'].values

array([[[  7,  41],
        [ 23,  98]],

       [[  3,  33],
        [  9,  90]],

       [[  5,  12],
        [ 10,  79]],

       [[  0,   6],
        [ 18,  87]],

       [[  5,  16],
        [ 34, 101]],

       [[  1,  25],
        [ 36,  82]],

       [[ 10,  20],
        [ 39,  78]],

       [[  2,  18],
        [ 14,  83]],

       [[ 12,  35],
        [ 26, 128]],

       [[  6,  21],
        [  6,  84]],

       [[ 15,  33],
        [ 39, 112]],

       [[ 12,  44],
        [ 27,  97]],

       [[  7,  51],
        [ 51, 103]],

       [[ 28,  48],
        [ 34,  85]],

       [[  2,  17],
        [ 17,  83]],

       [[ 11,  45],
        [ 27, 107]],

       [[ 15,  54],
        [ 29, 115]],

       [[ 27,  40],
        [ 35,  89]],

       [[ 11,  15],
        [ 25,  96]],

       [[  5,  12],
        [ 27,  94]],

       [[  3,  32],
        [ 19, 117]],

       [[  6,  32],
        [ 23, 129]],

       [[ 16,  53],
        [ 22, 126]],

       [[ 19,  26],
        [ 18, 

In [52]:
from pyproj import Proj, Transformer

# Define the projection
transformer = Transformer.from_crs('epsg:4326', 'epsg:5070')


def filter_coordinates(xarray, latlon):
    # Convert lat/lon to x/y
    x1, y1 = transformer.transform(latlon[1], latlon[0])  # min lat/lon
    x2, y2 = transformer.transform(latlon[3], latlon[2])  # max lat/lon
    
    mask = ((xarray.y >= y1) &
            (xarray.y <= y2) &
            (xarray.x >= x1) &
            (xarray.x <= x2))
    filtered_ds = xarray.where(mask, drop=True)
    return filtered_ds

In [53]:
# Define the states and their corresponding coordinates
states = {'Nebraska': nebraska, 'Kansas': kansas, 'Oklahoma': oklahoma,
          'Colorado': colorado, 'Wyoming': wyoming, 'Iowa': iowa, 
          'South Dakota': south_dakota, 'Missouri': missouri}  

# Create a line plot of 'stress_days' over time
fig = go.Figure()

# Iterate over the states
for state, coordinates in states.items():
    # Filter tmax_yearly for the current state
    tmax_yearly_state = filter_coordinates(tmax_yearly, coordinates)
    print(tmax_yearly_state)
    # Calculate the mean 'stress_days' for the current state
    stress_days_mean_state = tmax_yearly_state['stress_days'].mean(dim=['x', 'y'])
    
    # Add the data for the current state to the plot
    fig.add_trace(go.Scatter(x=stress_days_mean_state['time'].values, y=stress_days_mean_state.values, mode='lines', name=state))

# Set plot title and labels
fig.update_layout(title='Stress Days over Time', xaxis_title='Year', yaxis_title='Stress Days')

# Show the plot
fig.show()

<xarray.Dataset>
Dimensions:                  (time: 34, x: 0, y: 0)
Coordinates:
  * time                     (time) datetime64[ns] 1990-12-31 ... 2023-12-31
  * y                        (y) float32 
  * x                        (x) float32 
Data variables:
    lat                      (time, y, x) float32 dask.array<chunksize=(1, 0, 0), meta=np.ndarray>
    lambert_conformal_conic  (time, y, x) float64 
    lon                      (time, y, x) float32 dask.array<chunksize=(1, 0, 0), meta=np.ndarray>
    tmax                     (time, y, x) float32 dask.array<chunksize=(1, 0, 0), meta=np.ndarray>
    stress_days              (time, y, x) float64 dask.array<chunksize=(1, 0, 0), meta=np.ndarray>
<xarray.Dataset>
Dimensions:                  (time: 34, x: 0, y: 0)
Coordinates:
  * time                     (time) datetime64[ns] 1990-12-31 ... 2023-12-31
  * y                        (y) float32 
  * x                        (x) float32 
Data variables:
    lat                      (time,